### Enter full names of group members:

##### Name A: Vigdis-Irene Steinsund
##### Name B: Thomas Hasvold Adam

In [961]:
import math
import numpy as np
from sympy import prime
from pathlib import Path  # for paths of files
import csv
import copy
import random
from sklearn.metrics.pairwise import cosine_similarity

# ANSI escape codes for colors
class colors:
    red = '\033[91m'
    green = '\033[92m'
    blue = '\033[94m'
    end = '\033[0m'  


### 1. DGIM

#### 1.1. DGIM algorithm

In [962]:
# Default DGIM parameters

stream_path = 'data/my_stream.txt'

# The window size
N = 500 


In [963]:
def dgim_algorithm(stream_path, N):
    # Create the buckets and initialize the timestamp
    buckets = []
    timestamp = 0

    # Loop through the entire data stream, one bit at a time
    with open(stream_path) as f:
        while True:
            bit = f.read(1)

            # Clause to break while loop at the end of the stream
            if not bit:
                break

            # Update the timestamp - timestamp is recorded as modulo N
            timestamp = (timestamp + 1) % N

            # Tuple should be (size, timestamp)
            # If the bit is a '1', add new tuple to buckets with size = 1
            # Then start looping over all buckets to check if any need merging
            # If there are more than two tuples with the same size, merge them
            if bit == '1':
                new_bucket = (1, timestamp)
                buckets.append(new_bucket)
                for i in range(len(buckets)):
                    try:
                        # If there are three buckets with the same size, merge the first two
                        if(buckets[i][0] == buckets[i+1][0] == buckets[i+2][0]):
                            size = buckets[i][0]
                            # Merge the two buckets - we get double the bits
                            merged_size = size * 2 
                            timestamp_to_keep = buckets[i+1][1]
                            merged_tuple = (merged_size, timestamp_to_keep)
                            # Replace the second bucket with the merged one
                            buckets[i+1] = merged_tuple 
                            # Remove the first bucket
                            buckets.pop(i) 
                    except IndexError:
                        pass
            for bucket in buckets:
                # drop the last (oldest) bucket if its end-time is prior to N time units before the current time.
                if (timestamp - bucket[1]) % N >= N - 1:
                    buckets.remove(bucket)

    # Change to expected output format
    # Current: [(size, ts), (size, ts), ...]
    # Desired: List of timestamps for each size, so if there is only one '1' in the stream, the output should be [[ts]], and so on
    buckets_final = []
    ts_handled = []
    for i in range(len(buckets)):
        # If element is added, skip:
        if buckets[i][1] in ts_handled:
            continue
        # If element is last in list and not already added, add it to its own list:
        elif i == len(buckets)-1:
            buckets_final.append([buckets[i][1]])
            end_time_stamp = buckets[i][1]
            break
        # Check if the next element has the same size, if so add both to the same list
        elif(buckets[i][0] == buckets[i+1][0]):
            buckets_final.append([buckets[i][1], buckets[i+1][1]])
            ts_handled.append(buckets[i+1][1])
        else: # If ts not already added
            buckets_final.append([buckets[i][1]])
        
        ts_handled.append(buckets[i][1])
    
    buckets = buckets_final[::-1]
           
    return buckets, end_time_stamp



In [964]:
bucket = dgim_algorithm(stream_path, N)


In [965]:
print(f"The updated list of timestamps buckets from DGIM algorithm: \n {bucket[0]}")
print(f"The end timestamp: {bucket[1]}")   


The updated list of timestamps buckets from DGIM algorithm: 
 [[99], [91, 96], [83, 89], [63, 75], [44], [6], [321, 446], [188]]
The end timestamp: 99


#### 1.2. Query the Bucket 

In [966]:
def actual_count(stream_path, k):
    stream_list = []
    with open(stream_path, 'r') as file:
        for line in file:
            stream_list.extend(list(map(int, line.strip())))

    # Convert the list into a numpy array
    stream_array = np.array(stream_list)
    
    return int(np.sum(stream_array[-k:]))


In [967]:
import math
def dgim_query(bucket, N, k):

    # Extract the buckets and the end timestamp
    bucket_list, end_time_stamp = bucket

    # To estimate number of 1s in most recent k bits, we need to sum the sizes of the buckets (all but last) and add half the size of the last bucket.
    # We know the sorted list is in format [[ts, ts], [ts], [ts,ts], ...] where the size of the bucket is 2^index.
    # Therefore, first list in the bucket_list has size 2^0, second list has size 2^1, and so on.

    # Initialize the count of 1s
    one_count = 0

    # Buckets used
    buckets_used = []

    # Iterate through the buckets
    for bucket in bucket_list:
        for timestamp in bucket:
                # Check if timestamp falls within window
                if (end_time_stamp - timestamp) % N < k:
                    # If they do, add the size of the bucket to the count (from index which we know is 2^index)
                    one_count+= 2**bucket_list.index(bucket)
                    buckets_used.append(bucket)

    # Get the index of the last bucket
    last_bucket_index = bucket_list.index(buckets_used[-1])
    # Subtract half the size of the last bucket from total count
    one_count -= 2**(last_bucket_index)/2
        
    return math.ceil(one_count)


In [968]:
# List of queries
K = [10, 50, 100, 300, 500] 


In [969]:
print("---------------------------------------------------------------")
for k in K:
    dgim_count = dgim_query(bucket, 500, k)
    true_count = actual_count(stream_path, k)
    
    print(f"The total 1s in the last {k} bits by DGIM: {dgim_count}")
    print(f"The true count of 1s in the last {k} bits: {true_count}")
    print(f"The DGIM error for predicted 1s in the last {k} bits: \
    {round(abs(100*(dgim_count-true_count))/true_count,2)} %")
    print("---------------------------------------------------------------")


---------------------------------------------------------------
The total 1s in the last 10 bits by DGIM: 4
The true count of 1s in the last 10 bits: 5
The DGIM error for predicted 1s in the last 10 bits:     20.0 %
---------------------------------------------------------------
The total 1s in the last 50 bits by DGIM: 25
The true count of 1s in the last 50 bits: 26
The DGIM error for predicted 1s in the last 50 bits:     3.85 %
---------------------------------------------------------------
The total 1s in the last 100 bits by DGIM: 61
The true count of 1s in the last 100 bits: 51
The DGIM error for predicted 1s in the last 100 bits:     19.61 %
---------------------------------------------------------------
The total 1s in the last 300 bits by DGIM: 173
The true count of 1s in the last 300 bits: 150
The DGIM error for predicted 1s in the last 300 bits:     15.33 %
---------------------------------------------------------------
The total 1s in the last 500 bits by DGIM: 269
The true 

### 2. Bloom filters

In [970]:
# Username data for the creation of bloom filters - B
data_file = (Path("data/bloom_username").with_suffix('.csv'))

# Test data to check the functionality and false positive rate
test1_file = (Path("data/test1_username").with_suffix('.csv'))
test2_file = (Path("data/test2_username").with_suffix('.csv'))

# Default bloom filter parameters
bloom_size = 1500000 # parameter N
h = 3 # number of hash functions


In [971]:
# create an array of bloom filter with zeros
B = np.zeros(bloom_size)


In [972]:
B


array([0., 0., 0., ..., 0., 0., 0.])

#### 2.1. Create Bloom filter

In [973]:
import sympy

def generate_hash(h, N):
    hash_list = []

    # Generate a list of hash functions
    for i in range(h):
        # Generate a random prime number p
        p = sympy.randprime(2, 1000)
        # The hash function computes the sum of the ASCII values of the characters in the string multiplied by p^i mod N
        hash_function = lambda s, p=p, N=N: sum([ord(s[i]) * (p ** i) for i in range(len(s))]) % N
        # Append the hash function to the list
        hash_list.append(hash_function)
    
    return hash_list


In [974]:
hashes = generate_hash(h, bloom_size)


In [975]:
def create_bloom_filter(B, hashes, data):
    with data.open() as f:
        for name in f:
            
            # Update the hash index of the bloom filter with 1s
            for hash_function in hashes:
                index = hash_function(name)
                B[index] = 1
            
    return B


In [976]:
bloom_array = create_bloom_filter(B, hashes, data_file)


In [977]:
bloom_array


array([1., 1., 1., ..., 1., 1., 1.])

#### 2.2. Verify usernames

In [978]:
def single_verify_username(bloom_array, hashes, new_user):
    
    # Verify username and return a code of 0 or 1 (1 - username taken and 0 - username available)
    for hash_function in hashes:
        index = hash_function(new_user)
        # Check if bit is set or not
        if bloom_array[index] == 0:  
            return 0
    # If all bits are sets, username is likely already taken
    return 1 


In [979]:
# Feel free to test different usernames here

new_username = "KazeemTDT4305"

# new_username = "ShambaTDT4305"


In [980]:
user_code = single_verify_username(bloom_array, hashes, new_username)


In [981]:
if user_code == 1:
    print(colors.red + f"Username {new_username} has been taken. Try again!" + colors.end)
elif user_code == 0:
    print(colors.green + f"Username {new_username} is available. Congrats!" + colors.end)
else:
    print(colors.blue + f"Wrong pass code. Please reverify!" + colors.end)  


Username KazeemTDT4305 is available. Congrats!


In [982]:
def group_verify_username(bloom_array, hashes, data):
    # Initialize counts
    total_name = 0
    taken_name = 0
    
    with data.open() as f:
        for name in f:
            # Similar to the single verify, but returns a percentage of usernames taken...
            # ...(In other words seen already by the bloom filter during its creation)
            if single_verify_username(bloom_array, hashes, name) == 1:
                taken_name += 1  # Username likely seen/taken
            total_name += 1  # Increment total count regardless
            
    return round(taken_name/total_name*100,2)   


In [983]:
print("----------------------------------------------------------")
user_total = group_verify_username(bloom_array, hashes, test1_file)
print(f"Percentage of username seen before from test 1: {user_total}%")
print("----------------------------------------------------------")
user_total = group_verify_username(bloom_array, hashes, test2_file)
print(f"Percentage of username seen before from test 2: {user_total}%")
print("----------------------------------------------------------")


----------------------------------------------------------
Percentage of username seen before from test 1: 100.0%
----------------------------------------------------------
Percentage of username seen before from test 2: 23.81%
----------------------------------------------------------


### 3. Flajolet-Martin

In [1016]:
def flajolet_martin(input_stream):
    R = 0  # Initialize maximum rightmost zero bit position to 0

    # Define hash function h(x) = 6x + 1 mod 5
    hash_function = lambda x: (6*x + 1) % 5

    # Iterate over the input stream - for each element we want to find the number of trailing zeroes in the binary representation of the hash value
    for element in input_stream:
        # Apply hash function
        hash_value = hash_function(element)

        # Convert to binary after applying hash function
        binary_hash = bin(hash_value)[2:]

        # Calculate number of trailing zeroes in the binary hash
        trailing_zeroes = len(binary_hash) - len(binary_hash.rstrip('0'))

        # If the current number of trailing zeroes is greater than the current maximum, update the maximum
        R = max(R, trailing_zeroes)

    # Estimate the number of distinct elements
    distinct_estimate = 2 ** R

    return distinct_estimate


In [1017]:
# Input stream
input_stream1 = [1, 1, 2, 1, 2, 1, 1, 1, 1, 2, 1, 1]
input_stream2 = [1, 3, 2, 1, 2, 3, 4, 3, 1, 2, 3, 1]

# Run the Flajolet-Martin algorithm
distinct_estimate1 = flajolet_martin(input_stream1)
distinct_estimate2 = flajolet_martin(input_stream2)

# Print the estimated number of distinct elements
print("-----------------------------------------------------")
print(f"Distinct elements (estimated) in input stream 1:", distinct_estimate1)
print("-----------------------------------------------------")
print(f"Distinct elements (estimated) in input stream 2:", distinct_estimate2)
print("-----------------------------------------------------")


-----------------------------------------------------
Distinct elements (estimated) in input stream 1: 2
-----------------------------------------------------
Distinct elements (estimated) in input stream 2: 4
-----------------------------------------------------


### 4. Adword 

#### 4.1. Greedy Algorithm

In [986]:
# User queries
queries = ["big data", "big data", "big data","bloom filters", "bloom filters", "bloom filters",
           "flajolet martin", "flajolet martin", "flajolet martin", "dgim algorithm", "dgim algorithm", "dgim algorithm"]


In [987]:
# Company A B C and D keywords and budget $$$
global_companies = {
        'A': ["big data", "bloom filters", 3],
        'B': ["flajolet martin", 3],
        'C': ["flajolet martin", "dgim algorithm", 3],
        'D': ["big data", 3],
    }


In [988]:
def greedy_algorithm(local_companies, queries):
    # Initial revenue
    revenue = 0
    
    # To-do! update revenue using greedy algorithm
    
    return revenue


In [989]:
total_revenue = 0
total_trials = 10
print("Starting trials using Greedy Algorithm...")
print("------------------------------------------------")
for i in range(total_trials):
    local_companies = copy.deepcopy(global_companies)
    revenue = greedy_algorithm(local_companies, queries)
    total_revenue = total_revenue + revenue
    print(f"Trial {i+1} - Revenue generated: {revenue}")
print("------------------------------------------------")   
print("Average revenue generated for all trials: ",total_revenue/total_trials)


Starting trials using Greedy Algorithm...
------------------------------------------------
Trial 1 - Revenue generated: 0
Trial 2 - Revenue generated: 0
Trial 3 - Revenue generated: 0
Trial 4 - Revenue generated: 0
Trial 5 - Revenue generated: 0
Trial 6 - Revenue generated: 0
Trial 7 - Revenue generated: 0
Trial 8 - Revenue generated: 0
Trial 9 - Revenue generated: 0
Trial 10 - Revenue generated: 0
------------------------------------------------
Average revenue generated for all trials:  0.0


#### 4.2. Balance Algorithm

In [990]:
def balance_algorithm(local_companies, queries):
    # Initial revenue
    revenue = 0
    
    # To-do! update revenue using balance algorithm
    
    return revenue


In [991]:
total_revenue = 0
total_trials = 10
print("Starting trials using Balance Algorithm...")
print("-------------------------------------------")
for i in range(total_trials):
    local_companies = copy.deepcopy(global_companies)
    revenue = balance_algorithm(local_companies, queries)
    total_revenue = total_revenue + revenue
    print(f"Trial {i+1} - Revenue generated: {revenue}")
print("-------------------------------------------")   
print("Average revenue generated for all trials: ",total_revenue/total_trials)


Starting trials using Balance Algorithm...
-------------------------------------------
Trial 1 - Revenue generated: 0
Trial 2 - Revenue generated: 0
Trial 3 - Revenue generated: 0
Trial 4 - Revenue generated: 0
Trial 5 - Revenue generated: 0
Trial 6 - Revenue generated: 0
Trial 7 - Revenue generated: 0
Trial 8 - Revenue generated: 0
Trial 9 - Revenue generated: 0
Trial 10 - Revenue generated: 0
-------------------------------------------
Average revenue generated for all trials:  0.0


### 5. Recommender System

In [992]:
# Ratings matrix (each row corresponds to a movie, and each column corresponds to a user)
ratings_matrix = np.array([
    [1, 0, 3, 0, 0, 5, 0, 0, 5, 0, 4, 0],
    [0, 0, 5, 4, 0, 0, 4, 0, 0, 2, 1, 3],
    [2, 4, 0, 1, 2, 0, 3, 0, 4, 3, 5, 0],
    [0, 2, 4, 0, 5, 0, 0, 4, 0, 0, 2, 0],
    [0, 0, 4, 3, 4, 2, 0, 0, 0, 0, 2, 5],
    [1, 0, 3, 0, 3, 0, 0, 2, 0, 0, 4, 0]
])


#### 5.1. User-User Collaborative Filtering

In [993]:
def user_cf(rate_m, tup_mu, neigh):
    
    # To-do! implement a user-user CF using cosine similarity as distance measure
    
    return prediction   


In [994]:
# List of tuple of movie rating by users to be predicted e.g (1, 5) refers to the rating of movie 1 by user 5
list_mu_query = [(1, 5), (3, 3)]

# Neighbor selection (|N|)
neigh = 2


In [995]:
print("-----------------------------------------------------------------")   
for mu_query in list_mu_query:
    predicted_rating = user_cf(ratings_matrix, mu_query, neigh)
    print(f"The predicted rating of movie {mu_query[0]} by user {mu_query[1]}: {predicted_rating} (User-User CF)")
    print("-----------------------------------------------------------------")   


-----------------------------------------------------------------


NameError: name 'prediction' is not defined

#### 5.2. Item-Item Collaborative Filtering

In [ ]:
def item_cf(rate_m, tup_mu, neigh):
    
    # To-do! implement a item-item CF using cosine similarity as distance measure
    
    return prediction


In [ ]:
print("-----------------------------------------------------------------")   
for mu_query in list_mu_query:
    predicted_rating = item_cf(ratings_matrix, mu_query, neigh)
    print(f"The predicted rating of movie {mu_query[0]} by user {mu_query[1]}: {predicted_rating} (Item-Item CF)")
    print("-----------------------------------------------------------------")   


### Provide concise answers to all 5 cases in the Project 3 description below

#### Case 1

In [ ]:
# Enter answer here


#### Case 2

In [ ]:
# Enter answer here


#### Case 3

In [ ]:
# Enter answer here


#### Case 4

In [ ]:
# Enter answer here


#### Case 5

In [ ]:
# Enter answer here
